In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

Load dataframes and filter them.



In [37]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.loc[train.is_colon==1].drop(['is_colon', 'is_aca_cancer'],axis = 1)
train['is_non_cancer']=train['is_non_cancer'].astype('str')
test = test.loc[test.is_colon==1].drop(['is_colon', 'is_aca_cancer'],axis = 1)
test['is_non_cancer']=test['is_non_cancer'].astype('str')
train

,filepath,is_non_cancer
0,data\colon_image_sets\colon_n\colonn3528.jpeg,True
1,data\colon_image_sets\colon_n\colonn3115.jpeg,True
2,data\colon_image_sets\colon_n\colonn3012.jpeg,True
3,data\colon_image_sets\colon_n\colonn4184.jpeg,True
4,data\colon_image_sets\colon_n\colonn2768.jpeg,True
...,...,...
7995,data\colon_image_sets\colon_aca\colonca2665.jpeg,False
7996,data\colon_image_sets\colon_aca\colonca1066.jpeg,False
7997,data\colon_image_sets\colon_aca\colonca3033.jpeg,False
7998,data\colon_image_sets\colon_aca\colonca1014.jpeg,False


In [28]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

target_size = (256,256)

In [39]:
train_datagen.flow_from_dataframe(train,
                                  x_col='filepath',
                                  y_col='is_non_cancer',
                                  target_size=target_size,
                                  batch_size=40,
                                  class_mode='binary')
test_datagen.flow_from_dataframe(test,
                                  x_col='filepath',
                                  y_col='is_non_cancer',
                                  target_size=target_size,
                                  batch_size=40,
                                  class_mode='binary')

Found 8000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
